In [1]:
import pandas as pd
import numpy as np
import datetime
from itertools import product
from scipy.sparse.linalg import svds

#from surprise import Reader, Dataset, evaluate, print_perf, GridSearch
#from surprise import SVD, SVDpp, Prediction, accuracy
#from sklearn.metrics import roc_auc_score

In [2]:
data = pd.read_csv('data_engineered_features500.csv', nrows=60018)
#data = data.query("userid == ['user_000001','user_000002','user_000003','user_000004','user_000005']")

#data = data[['userid', 'track-name', 'artist-name', 'weekend', 'skipped']]
data.rename(columns={'track-name':'track',
                     'artist-name':'artist',
                     'track-total-count':'trackcount',
                     'track-weekday-daytime-count':'trackweekdaydaytimecount',
                     'artist-total-count':'artistcount',
                     'artist-weekday-daytime-count':'artistweekdaydaytimecount'}, inplace=True)

data.drop(['timestamp', 'songlength', 'last-seen-song', 'last-seen-artist', 'gender', 'age'], axis=1, inplace=True)
data = data[[c for c in data if c not in ['skipped']] + ['skipped']]

In [4]:
print(len(data))
data.dtypes

60018


userid                       object
track                        object
artist                       object
weekday                       int64
hour                          int64
weekend                       int64
daytime                       int64
trackcount                    int64
trackweekdaydaytimecount      int64
month                         int64
quarter                       int64
artistcount                   int64
artistweekdaydaytimecount     int64
skipped                       int64
dtype: object

In [47]:
df = data[['userid', 'track', 'artist', 'weekend', 'skipped']].copy()
df['trackartist'] = df['track']+'___'+df['artist']
df.drop(['track', 'artist'], axis=1, inplace=True)

df = df.groupby(['userid', 'trackartist', 'weekend'], as_index=False).mean()

In [48]:
distinct_df = df.drop('skipped', axis=1).drop_duplicates()

temp = pd.DataFrame(list(product(distinct_df['userid'].drop_duplicates(),
                                 distinct_df['trackartist'].drop_duplicates(),
                                 distinct_df['weekend'].drop_duplicates())),
                   columns = ['userid', 'trackartist', 'weekend'])
len(temp)

24500

In [53]:
new_data = temp.merge(df, how='left',
                      on=['userid', 'trackartist', 'weekend']) \
               .fillna(1)

del

In [54]:
new_data.describe()

,weekend,skipped
count,24500.00000,24500.000000
mean,0.50000,0.805027
std,0.50001,0.392899
min,0.00000,0.000000
25%,0.00000,1.000000
50%,0.50000,1.000000
75%,1.00000,1.000000
max,1.00000,1.000000
